In [1]:
import h2o
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from h2o.estimators import H2ORandomForestEstimator

h2o.init()

db = pd.read_csv('dataset/db_modelli.csv').drop(columns = 'Unnamed: 0')
# codifica
db['conc1_mean'] = np.where(db['conc1_mean'].values > 1, 1, 0)

target = 'conc1_mean'
predictors = db.columns.to_list()
predictors.remove(target)
predictors.remove('test_cas')

# categorical to factors
db_h = h2o.H2OFrame(db)
# occorre
for i in db.columns:
    if db[i].dtypes == 'object':
        db_h[i] = db_h[i].asfactor()

db_h['conc1_mean'] = db_h['conc1_mean'].asfactor()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.6+8-b765.1, mixed mode)
  Starting server from C:\Users\Matteo\anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Matteo\AppData\Local\Temp\tmph_wao74h
  JVM stdout: C:\Users\Matteo\AppData\Local\Temp\tmph_wao74h\h2o_Matteo_started_from_python.out
  JVM stderr: C:\Users\Matteo\AppData\Local\Temp\tmph_wao74h\h2o_Matteo_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,08 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.6
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_Matteo_4w3le3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.973 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
# Griglia da utilizzare
ntrees_grid = [10, 30, 60, 90, 120, 150, 200, 300, 500, 700, 1000]
ntrees_grid 

[10, 30, 60, 90, 120, 150, 200, 300, 500, 700, 1000]

In [6]:
# splitting
train, test = db_h.split_frame(ratios = [0.7])

mod = H2ORandomForestEstimator(nfolds = 3, seed = 1234, ntrees = ntrees_grid[0],
                               categorical_encoding = 'one_hot_explicit',col_sample_rate_change_per_level = 1.08,
                               col_sample_rate_per_tree = 0.69, histogram_type = 'RoundRobin',
                               max_depth = 20, min_rows = 2, min_split_improvement = 1e-8,
                               nbins= 512, nbins_cats = 1024, sample_rate = 0.85)

mod.train(x = predictors, y = target, training_frame = train)

mod.model_performance(test)

drf Model Build progress: |████████████████████████████████████████████ (cancelled)  95%


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_b49bbb0afb57df67f6c9930e96784f2a> was cancelled by the user.